Load the shareable library (note versioned file from runtime package, not development package):

In [ ]:
import ctypes as ct

cairo = ct.cdll.LoadLibrary("libcairo.so.2")

Note the use of the explicitly-versioned library name `libcairo.so.2` instead of `libcairo.so`; latter comes from development package, former from run-time package. Development package should only be needed for building compiled code (e.g. in C) against the library; for users who only need to use your module to run Python code, the run-time package should be sufficient.

**API Versus ABI** The versioning of the shared library file is to deal with incompatible changes in generated code in client programs. For example, a structure layout might change, without requiring any changes to client _source_ code: but once the client app is compiled against the new layout, it will not work against the old one, and vice versa. So existing code cannot load the new version of the library, it must continue to load the old one, while newly-built code uses the new library.

Representation of [`cairo_matrix_t`](https://www.cairographics.org/manual/cairo-cairo-matrix-t.html#cairo-matrix-t) (also add a `__repr__` for easy debugging):

In [ ]:
class matrix_t(ct.Structure) :
    _fields_ = \
        [
            ("xx", ct.c_double),
            ("yx", ct.c_double),
            ("xy", ct.c_double),
            ("yy", ct.c_double),
            ("x0", ct.c_double),
            ("y0", ct.c_double),
        ]

    def __repr__(self) :
        return "[" + ", ".join(repr(getattr(self, f[0])) for f in matrix_t._fields_) + "]"
    #end __repr__
#end matrix_t

Define routine prototypes, e.g.

In [ ]:
cairo.cairo_matrix_init_identity.restype = None
cairo.cairo_matrix_init_identity.argtypes = (ct.POINTER(matrix_t),)

Example creation of C object:

In [ ]:
m = matrix_t()
m

Pass to library routine and observe result:

In [ ]:
cairo.cairo_matrix_init_identity(ct.byref(m))
m

Try another routine:

In [ ]:
cairo.cairo_matrix_scale.restype = None
cairo.cairo_matrix_scale.argtypes = (ct.POINTER(matrix_t), ct.c_double, ct.c_double)


In [ ]:
cairo.cairo_matrix_scale(m, 3, 2)
m

Note how you could pass straight Python numeric expressions for `c_double` args; conversion for such simple types is automatic.

In [ ]:
libc = ct.cdll.LoadLibrary("libc.so.6")

class FILE(ct.Structure) :
    _fields_ = []
#end FILE

stderr = ct.POINTER(FILE).in_dll(libc, "stderr")


libc.fprintf.argtypes = (ct.POINTER(FILE), ct.c_char_p,)

In [ ]:
libc.fprintf(stderr, "hello, world!".encode())